In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema # Importing necessary classes

# Load environment variables from a .env file. 🌍
# This securely loads API keys (e.g., HUGGINGFACEHUB_API_TOKEN) from an environment file.
load_dotenv()

True

In [2]:
# Define the HuggingFaceEndpoint model. 🤖
# This connects to a specific large language model hosted on the Hugging Face Inference API.
# 'google/gemma-2-2b-it' is a smaller instruction-tuned model.
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

In [3]:
# Wrap the HuggingFaceEndpoint with ChatHuggingFace. 💬
# This adapts the raw Hugging Face model for LangChain's chat interface.
model = ChatHuggingFace(llm=llm)

In [4]:
# Define the schema for the desired structured output. 📝
# ResponseSchema is used by StructuredOutputParser to define the expected fields
# and their descriptions.
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

In [5]:
# Initialize the StructuredOutputParser from the defined schema. ⚙️
# This parser will generate formatting instructions for the LLM and
# then attempt to parse the LLM's raw text output into a Python dictionary
# conforming to the 'schema'.
parser = StructuredOutputParser.from_response_schemas(schema)

In [6]:
# Define the prompt template. ✍️
# It includes a placeholder for the 'topic' and importantly,
# `format_instruction`: This will be populated by the parser with a detailed
# text-based instruction on how the LLM should format its output (e.g., as JSON).
template = PromptTemplate(
    template='Give 3 facts about {topic}\n{format_instruction}', # Corrected newline
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

In [7]:
# Create a LangChain Expression Language (LCEL) chain. 🔗
# 1. `template`: Generates the prompt, including formatting instructions.
# 2. `model`: Sends the prompt to the HuggingFace LLM and gets a text response.
# 3. `parser`: Attempts to parse the raw text response from the LLM into a dictionary
#    based on the `schema`.
chain = template | model | parser

In [8]:
# Invoke the chain with the topic 'black hole'. 🚀
# The LLM will receive instructions to output 3 facts in a specific structured format.
# However, `google/gemma-2-2b-it` (and similar smaller open-source models) are
# generally not reliably able to produce perfectly structured JSON from text-based
# formatting instructions. They often generate free-form text, which will cause
# the `StructuredOutputParser` to fail or produce an incomplete result.
result = chain.invoke({'topic':'black hole'})

# Print the result. 📊
# If the parsing is successful, it will print a dictionary like:
# {'fact_1': '...', 'fact_2': '...', 'fact_3': '...'}
# However, it's highly probable that this will either:
# 1. Raise a parsing error if the LLM's output is not valid JSON.
# 2. Return an empty or incomplete dictionary if the parser can't find the structure.
# You'd typically need more robust error handling or use models with native structured output capabilities
# for reliable results with this pattern.
print(result)

{'fact_1': 'Black holes are regions in spacetime where gravity is so strong that nothing, not even light, can escape.', 'fact_2': 'They are formed when very massive stars die and collapse under their own gravity.', 'fact_3': 'While black holes are invisible, scientists can detect them through their gravitational effects on nearby objects, like stars orbiting them.'}
